In [3]:
import pandas as pd
import pandas as pd

from sklearn.model_selection import train_test_split

from kmodes.kmodes import KModes
from sklearn.model_selection import train_test_split
from pycaret.classification import *

%run class/patient_details.py
%run util/utils.py


Cardiovascular Disease Predictor

In [4]:
# Cardiovascular
cardio_df = pd.read_csv("data/cardio_clean.csv")

df_male = cardio_df.query("gender == 1")
df_female = cardio_df.query("gender == 0")

# female data
# Building the model with using K-Mode with "Huang" initialization
km_huang = KModes(n_clusters=2, init="Huang", n_init=5, verbose=0)

male_kmode = km_huang.fit(df_male)
female_kmode = km_huang.fit(df_female)

clusters_female = km_huang.fit_predict(df_female)
clusters_male = km_huang.fit_predict(df_male)

# female DataFrame with Clusters
df_female.insert(0, "cluster", clusters_female, True)

# male DataFrame with Clusters
df_male.insert(0, "cluster", clusters_male, True)

# replacing cluster column values to merge dataframes after
df_male["cluster"].replace({0: 2, 1: 3}, inplace=True)

# merging female and male data
df_clusters = pd.concat([df_female, df_male], ignore_index=True, sort=False)

clf1 = setup(data=df_clusters,
             target='cardio',
             preprocess=False,
             silent=True)

models()

top5_cvd = compare_models(sort='AUC',
                          n_select=5,
                          )
stack_model_cvd = stack_models(
    estimator_list=top5_cvd, meta_model=top5_cvd[0], optimize='AUC')

final_model_cvd = finalize_model(stack_model_cvd)


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8632,0.9469,0.8229,0.8879,0.8542,0.7257,0.7275
1,0.8577,0.9403,0.8278,0.8734,0.8500,0.7149,0.7158
2,0.8720,0.9491,0.8322,0.8974,0.8636,0.7433,0.7452
3,0.8660,0.9468,0.8288,0.8886,0.8576,0.7314,0.7330
4,0.8539,0.9439,0.8137,0.8774,0.8443,0.7071,0.7089
5,0.8663,0.9490,0.8337,0.8850,0.8586,0.7320,0.7331
6,0.8603,0.9474,0.8337,0.8737,0.8532,0.7201,0.7209
7,0.8675,0.9484,0.8507,0.8737,0.8621,0.7346,0.7348
8,0.8580,0.9476,0.8254,0.8758,0.8498,0.7153,0.7164


Diabetes Predictor

In [5]:
# Diabetes
diabetes_df = pd.read_csv('data/diabetes_clean.csv')
diabetes_df.info()

X = diabetes_df.iloc[:, :-1]
y = diabetes_df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42)

clf2 = setup(data=diabetes_df,
             target='Outcome',
             preprocess=False,
             silent=True)

models()

top5_diabetes = compare_models(sort='AUC', n_select=5)

blend_soft_diabetes = blend_models(
    estimator_list=top5_diabetes, optimize='AUC', method='soft')

cali_model_diabetes = calibrate_model(blend_soft_diabetes)

final_model_diabetes = finalize_model(cali_model_diabetes)


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7037,0.8075,0.4737,0.6000,0.5294,0.3175,0.3223
1,0.7593,0.8135,0.5789,0.6875,0.6286,0.4524,0.4561
2,0.7593,0.8316,0.5789,0.6875,0.6286,0.4524,0.4561
3,0.8519,0.8767,0.6316,0.9231,0.7500,0.6499,0.6736
4,0.7963,0.8827,0.4737,0.9000,0.6207,0.4992,0.5472
5,0.8333,0.8406,0.5263,1.0000,0.6897,0.5902,0.6470
6,0.7778,0.8722,0.8421,0.6400,0.7273,0.5456,0.5602
7,0.8113,0.8508,0.6111,0.7857,0.6875,0.5554,0.5644
8,0.8113,0.8714,0.6667,0.7500,0.7059,0.5677,0.5698


Reusable Functions

In [38]:
def predict_diabetes(patient_details):
    prediction_diabetes = final_model_diabetes.predict([[
        patient_details.pregnancies,
        patient_details.glucose,
        patient_details.diastolic_bp,
        patient_details.skin_thickness,
        patient_details.insulin,
        patient_details.bmi,
        patient_details.dpf,
        patient_details.age
    ]])
    return prediction_diabetes[0]


def predict_cvd(patient_details):
    prediction_cvd = final_model_cvd.predict([[
        patient_details.cluster,
        patient_details.gender,
        patient_details.age_class,
        patient_details.bmi_class,
        patient_details.map_class,
        patient_details.cholesterol_class,
        patient_details.glucose_class,
        patient_details.smoke,
        patient_details.active
    ]])
    return prediction_cvd[0]


def assign_cluster_male(patient):
    cluster = male_kmode.predict(patient)
    if cluster[0] == 0:
        return 2
    if cluster[0] == 1:
        return 3


def assign_cluster_female(patient):
    predict = female_kmode.predict(patient)
    return predict[0]


def assign_cluster(patient_details):
    if patient_details.gender == 0:  # Female
        return assign_cluster_female([[
            patient_details.gender,
            patient_details.age_class,
            patient_details.bmi_class,
            patient_details.map_class,
            patient_details.cholesterol_class,
            patient_details.glucose_class,
            patient_details.smoke,
            patient_details.active,
            patient_details.cvd_prediction
        ]])
    else:
        return assign_cluster_male([[
            patient_details.gender,
            patient_details.age_class,
            patient_details.bmi_class,
            patient_details.map_class,
            patient_details.cholesterol_class,
            patient_details.glucose_class,
            patient_details.smoke,
            patient_details.active,
            patient_details.cvd_prediction
        ]])

def proccess_patient_details(age, gender, pregnancies, glucose, cholesterol, insulin, systolic_bp, diastolic_bp, skin_thickness, bmi, smoke, active, dpf):
    patient_details = Patient()

    patient_details.age = age
    patient_details.gender = gender
    patient_details.pregnancies = pregnancies
    patient_details.glucose = glucose
    patient_details.cholesterol = cholesterol
    patient_details.insulin = insulin
    patient_details.systolic_bp = systolic_bp
    patient_details.diastolic_bp = diastolic_bp
    patient_details.skin_thickness = skin_thickness
    patient_details.bmi = bmi
    patient_details.smoke = smoke
    patient_details.active = active
    patient_details.dpf = dpf

    #Initial value, might change after prediction
    patient_details.cvd_prediction = 0
    patient_details.diabetes_prediction = 0

    patient_details = process_patient_classification(patient_details)

    return patient_details

def process_patient_classification(patient_details):
    patient_details.mean_arterial_pressure = compute_mean_arterial_pressure(patient_details.systolic_bp, patient_details.diastolic_bp)
    patient_details.age_class = get_age_class(patient_details.age)
    patient_details.bmi_class = get_bmi_class(patient_details.bmi)
    patient_details.map_class = get_map_class(patient_details.mean_arterial_pressure)
    patient_details.cholesterol_class = get_cholesterol_class(patient_details.cholesterol)
    patient_details.glucose_class = get_glucose_class(patient_details.glucose)

    patient_details.cluster = assign_cluster(patient_details)

    patient_details.cvd_prediction = predict_cvd(patient_details)
    patient_details.diabetes_prediction = predict_diabetes(patient_details)

    return patient_details



Web Application Implementation

In [39]:
import json
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([
    html.Div([
        html.H3(children='Cardiovascular Disease and Diabetes Prediction',
                className='page-title'),
    ]),
    html.Hr(),
    dbc.Row([
        dbc.Col([
            dbc.Card([

                html.Div(
                    id="feedback-container",
                    children=[
                    ]),

                html.Div([
                    dbc.Label("Age"),
                    dbc.Input(id="age", type="number",  min=0,
                              placeholder="Enter age"),
                ], className='form-container mb-3'),

                html.Div([
                    dbc.Label("Gender"),
                    dcc.Dropdown(
                        id="gender",
                        options=[
                            {'label': 'Male', 'value': 1, },
                            {'label': 'Female', 'value': 0}
                        ],
                        placeholder="Please select"
                    ),
                ], className='form-container mb-3'),

                html.Div([
                    dbc.Label("Number of Pregnancies"),
                    dbc.Input(id="pregnancies", type="number",  min=0,
                              placeholder="Enter number of pregnancies"),
                ], className='form-container mb-3'),

                html.Div([
                    dbc.Label("Glucose Level"),
                    dbc.Input(id="glucose", type="number",  min=0,
                              placeholder="Enter glucose level"),
                ], className='form-container mb-3'),

                html.Div([
                    dbc.Label("Cholesterol Level"),
                    dbc.Input(id="cholesterol", type="number",  min=0,
                              placeholder="Enter cholesterol level"),
                ], className='form-container mb-3'),

                html.Div([
                    dbc.Label("Insulin Level"),
                    dbc.Input(id="insulin", type="number",  min=0,
                              placeholder="Enter insulin level"),
                ], className='form-container mb-3'),

                html.Div([
                    dbc.Label("Systolic Blood Pressure"),
                    dbc.Input(id="systolic_bp", type="number",  min=0,
                              placeholder="Enter blood pressure here"),
                ], className='form-container mb-3'),

                html.Div([
                    dbc.Label("Diastolic Blood Pressure"),
                    dbc.Input(id="diastolic_bp", type="number", min=0,
                              placeholder="Enter blood pressure here"),
                ], className='form-container mb-3'),

                html.Div([
                    dbc.Label("Skin Thickness"),
                    dbc.Input(id="skin_thickness", type="number",
                              min=0, placeholder="Enter thickness"),
                ], className='form-container mb-3'),

                html.Div([
                    dbc.Label("Body Mass Index (BMI)"),
                    dbc.Input(id="bmi", type="number",  min=0,
                              placeholder="Enter BMI"),
                ], className='form-container mb-3'),

                html.Div([
                    dbc.Label("Do You Smoke?"),
                    dcc.Dropdown(
                        id="smoke",
                        options=[
                            {'label': 'Yes', 'value': 1},
                            {'label': 'No', 'value': 0}
                        ],
                        placeholder="Please select"
                    ),
                ], className='form-container mb-3'),

                html.Div([
                    dbc.Label("Are You Physically Active?"),
                    dcc.Dropdown(
                        id="active",
                        options=[
                            {'label': 'Yes', 'value': 1},
                            {'label': 'No', 'value': 0}
                        ],
                        placeholder="Please select"
                    ),
                ], className='form-container mb-3'),

                html.Div([
                    dbc.Label("Does your family have a history of diabetes?"),
                    dcc.Dropdown(
                        id="dpf",
                        options=[
                            {'label': 'Yes', 'value': 1},
                            {'label': 'No', 'value': 0}
                        ],
                        placeholder="Please select"
                    ),
                ], className='form-container mb-3'),
            ], body=True,
                style={
                'margin-top': '0.5rem',
                'margin-bottom': '1rem',
            }),

            html.Div([
                dbc.Button('Predict', id='submit-val',
                           color='primary', className='w-100 mb-3'),
            ]),

        ], md=5),


        dbc.Col([
            dbc.Card([
                html.Div(
                    id="output-container",
                    children=[
                    ]),
            ], body=True)
        ], md=7),
    ],
        align="center",
    ),
],
    fluid=True,
)

@app.callback(
    Output(component_id='feedback-container', component_property='children'),
    Input(component_id='submit-val', component_property='n_clicks'),
    Input(component_id='age', component_property='value'),
    Input(component_id='gender', component_property='value'),
    Input(component_id='glucose', component_property='value'),
    Input(component_id='cholesterol', component_property='value'),
    Input(component_id='systolic_bp', component_property='value'),
    Input(component_id='diastolic_bp', component_property='value'),
    Input(component_id='pregnancies', component_property='value'),
    Input(component_id='skin_thickness', component_property='value'),
    Input(component_id='insulin', component_property='value'),
    Input(component_id='bmi', component_property='value'),
    Input(component_id='dpf', component_property='value'),
    Input(component_id='smoke', component_property='value'),
    Input(component_id='active', component_property='value')
)

def send_feedback(n_clicks, age, gender, glucose, pregnancies, cholesterol, systolic_bp, diastolic_bp, skin_thickness, insulin, bmi, dpf, smoke, active):
    if (n_clicks is not None and
        (age is None
        or gender is None
        or pregnancies is None
        or glucose is None
        or cholesterol is None
        or systolic_bp is None
        or diastolic_bp is None
        or skin_thickness is None
        or insulin is None
        or bmi is None
        or dpf is None
        or smoke is None
        or active is None)):
        return dbc.Alert('All fields are required', color='danger')

@app.callback(
    Output(component_id='output-container', component_property='children'),
    Input(component_id='submit-val', component_property='n_clicks'),
    Input(component_id='age', component_property='value'),
    Input(component_id='gender', component_property='value'),
    Input(component_id='pregnancies', component_property='value'),
    Input(component_id='glucose', component_property='value'),
    Input(component_id='cholesterol', component_property='value'),
    Input(component_id='insulin', component_property='value'),
    Input(component_id='systolic_bp', component_property='value'),
    Input(component_id='diastolic_bp', component_property='value'),
    Input(component_id='skin_thickness', component_property='value'),
    Input(component_id='bmi', component_property='value'),
    Input(component_id='smoke', component_property='value'),
    Input(component_id='active', component_property='value'),
    Input(component_id='dpf', component_property='value')
)

def update_output(n_clicks, age, gender, pregnancies, glucose, cholesterol,  insulin, systolic_bp, diastolic_bp, skin_thickness, bmi, smoke, active, dpf):
    if (n_clicks is not None
        and age is not None
        and gender is not None
        and pregnancies is not None
        and glucose is not None
        and cholesterol is not None
        and insulin is not None
        and systolic_bp is not None
        and diastolic_bp is not None
        and skin_thickness is not None
        and bmi is not None
        and smoke is not None
        and active is not None
        and dpf is not None):
        
        patient = proccess_patient_details(age, gender, pregnancies, glucose, cholesterol, insulin, systolic_bp, diastolic_bp, skin_thickness, bmi, smoke, active, dpf)

        patient_map = {}

        patient_map["patient"] = patient.map_patient_details()
        
        patient_map["health_statuses"] = get_health_status(patient_map["patient"]["bmi_class"], patient_map["patient"]["map_class"], patient_map["patient"]["cholesterol_class"], patient_map["patient"]["glucose_class"], patient_map["patient"]["cvd_prediction"], patient_map["patient"]["diabetes_prediction"])
        
        print(patient_map)

        string = json.dumps(patient_map)

        return string


if __name__ == '__main__':
    app.run_server(port=4050)


Dash is running on http://127.0.0.1:4050/

Dash is running on http://127.0.0.1:4050/

Dash is running on http://127.0.0.1:4050/

Dash is running on http://127.0.0.1:4050/

Dash is running on http://127.0.0.1:4050/

Dash is running on http://127.0.0.1:4050/

Dash is running on http://127.0.0.1:4050/

Dash is running on http://127.0.0.1:4050/

Dash is running on http://127.0.0.1:4050/

Dash is running on http://127.0.0.1:4050/

Dash is running on http://127.0.0.1:4050/

Dash is running on http://127.0.0.1:4050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:4050 (Press CTRL+C to quit)
127.0.0.1 - - [16/May/2022 22:56:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2022 22:56:26] "GET /assets/header.css?m=1652455464.7393887 HTTP/1.1" 304 -
127.0.0.1 - - [16/May/2022 22:56:26] "GET /assets/style.css?m=1652453253.2719538 HTTP/1.1" 304 -
127.0.0.1 - - [16/May/2022 22:56:26] "GET /assets/typography.css?m=1652437109.3968763 HTTP/1.1" 304 -
127.0.0.1 - - [16/May/2022 22:56:26] "GET /_dash-component-suites/dash/dcc/dash_core_components-shared.js.map HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2022 22:56:26] "GET /_dash-component-suites/dash/dcc/dash_core_components.js.map HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2022 22:56:26] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2022 22:56:26] "GET /_dash-component-suites/dash/html/dash_html_components.min.js.map HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2022 22:56:26] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2022 22:56:26] "GET /_dash-component-suites

{'patient': {'cluster': '1', 'age': '31', 'gender': '0', 'cholesterol': '236', 'systolic_bp': '143', 'diastolic_bp': '93', 'mean_arterial_pressure': '109.66666666666667', 'skin_thickness': '27', 'insulin': '200', 'bmi': '29', 'dpf': '0', 'age_class': '0', 'bmi_class': '2', 'map_class': '2', 'cholesterol_class': '1', 'glucose_class': '2', 'smoke': '0', 'active': '0', 'cvd_prediction': '1', 'diabetes_prediction': '0'}, 'health_statuses': {'bmi_status': 'Overweight', 'map_status': 'Elevated', 'cholesterol_status': 'Borderline High', 'glucose_status': 'High', 'cvd_risk': 'High Risk', 'diabetes_risk': 'Low Risk'}}


127.0.0.1 - - [16/May/2022 22:57:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2022 22:57:50] "POST /_dash-update-component HTTP/1.1" 200 -


{'patient': {'cluster': '1', 'age': '31', 'gender': '0', 'cholesterol': '236', 'systolic_bp': '143', 'diastolic_bp': '93', 'mean_arterial_pressure': '109.66666666666667', 'skin_thickness': '27', 'insulin': '200', 'bmi': '29', 'dpf': '1', 'age_class': '0', 'bmi_class': '2', 'map_class': '2', 'cholesterol_class': '1', 'glucose_class': '2', 'smoke': '0', 'active': '0', 'cvd_prediction': '1', 'diabetes_prediction': '0'}, 'health_statuses': {'bmi_status': 'Overweight', 'map_status': 'Elevated', 'cholesterol_status': 'Borderline High', 'glucose_status': 'High', 'cvd_risk': 'High Risk', 'diabetes_risk': 'Low Risk'}}


...

{'patient': {'cluster': '1', 'age': '29', 'gender': '0', 'cholesterol': '101', 'systolic_bp': '131', 'diastolic_bp': '86', 'mean_arterial_pressure': '101.0', 'skin_thickness': '30', 'insulin': '164', 'bmi': '29', 'dpf': '0', 'age_class': '0', 'bmi_class': '2', 'map_class': '2', 'cholesterol_class': '0', 'glucose_class': '1', 'smoke': '0', 'active': '1', 'cvd_prediction': '1', 'diabetes_prediction': '0'}, 'health_statuses': {'bmi_status': 'Overweight', 'map_status': 'Elevated', 'cholesterol_status': 'Normal', 'glucose_status': 'Borderline High', 'cvd_risk': 'High Risk', 'diabetes_risk': 'Low Risk'}}
{'patient': {'cluster': '0', 'age': '29', 'gender': '0', 'cholesterol': '101', 'systolic_bp': '130', 'diastolic_bp': '86', 'mean_arterial_pressure': '100.66666666666667', 'skin_thickness': '30', 'insulin': '164', 'bmi': '29', 'dpf': '0', 'age_class': '0', 'bmi_class': '2', 'map_class': '1', 'cholesterol_class': '0', 'glucose_class': '1', 'smoke': '0', 'active': '1', 'cvd_prediction': '0', 'd

127.0.0.1 - - [16/May/2022 23:02:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2022 23:02:41] "POST /_dash-update-component HTTP/1.1" 200 -


{'patient': {'cluster': '1', 'age': '29', 'gender': '0', 'cholesterol': '101', 'systolic_bp': '131', 'diastolic_bp': '86', 'mean_arterial_pressure': '101.0', 'skin_thickness': '30', 'insulin': '164', 'bmi': '29', 'dpf': '0', 'age_class': '0', 'bmi_class': '2', 'map_class': '2', 'cholesterol_class': '0', 'glucose_class': '1', 'smoke': '0', 'active': '1', 'cvd_prediction': '1', 'diabetes_prediction': '0'}, 'health_statuses': {'bmi_status': 'Overweight', 'map_status': 'Elevated', 'cholesterol_status': 'Normal', 'glucose_status': 'Borderline High', 'cvd_risk': 'High Risk', 'diabetes_risk': 'Low Risk'}}


127.0.0.1 - - [16/May/2022 23:02:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2022 23:02:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2022 23:02:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2022 23:02:52] "POST /_dash-update-component HTTP/1.1" 200 -


{'patient': {'cluster': '1', 'age': '29', 'gender': '0', 'cholesterol': '101', 'systolic_bp': '132', 'diastolic_bp': '86', 'mean_arterial_pressure': '101.33333333333333', 'skin_thickness': '30', 'insulin': '164', 'bmi': '29', 'dpf': '0', 'age_class': '0', 'bmi_class': '2', 'map_class': '2', 'cholesterol_class': '0', 'glucose_class': '1', 'smoke': '0', 'active': '1', 'cvd_prediction': '1', 'diabetes_prediction': '0'}, 'health_statuses': {'bmi_status': 'Overweight', 'map_status': 'Elevated', 'cholesterol_status': 'Normal', 'glucose_status': 'Borderline High', 'cvd_risk': 'High Risk', 'diabetes_risk': 'Low Risk'}}
{'patient': {'cluster': '1', 'age': '29', 'gender': '0', 'cholesterol': '101', 'systolic_bp': '133', 'diastolic_bp': '86', 'mean_arterial_pressure': '101.66666666666667', 'skin_thickness': '30', 'insulin': '164', 'bmi': '29', 'dpf': '0', 'age_class': '0', 'bmi_class': '2', 'map_class': '2', 'cholesterol_class': '0', 'glucose_class': '1', 'smoke': '0', 'active': '1', 'cvd_predict